# 🧳 Plan your trip with Kayak 🧳
The marketing team needs help on a new project. After doing some user research, the team discovered that 70% of their users who are planning a trip would like to have more information about the destination they are going to.

In addition, user research shows that people tend to be defiant about the information they are reading if they don't know the brand which produced the content.

Therefore, Kayak Marketing Team would like to create an application that will recommend where people should plan their next holidays. The application should be based on real data about:

* Weather
* Hotels in the area
* The application should then be able to recommend the best destinations and hotels based on the above variables at any given time.

## 🎯 Goals 
As the project has just started, your team doesn't have any data that can be used to create this application. Therefore, your job will be to:

* Scrape data from destinations
* Get weather data from each destination
* Get hotels' info about each destination
* Store all the information above in a data lake
* Extract, transform and load cleaned data from your datalake to a data warehouse

## 📚 Imports

In [ ]:
import os 
import logging
import scrapy
from scrapy.crawler import CrawlerProcess
import requests
import pandas as pd
import json
import numpy as np
import warnings
warnings.filterwarnings("ignore")

## 🌧️ Get weather data with an API
Marketing team wants to focus first on the best cities to travel to in France. According One Week In.com here are the top-35 cities to visit in France :

In [ ]:
cities = ["Mont Saint Michel", "St Malo", "Bayeux", "Le Havre", "Rouen", "Paris", "Amiens", "Lille", "Strasbourg", "Chateau du Haut Koenigsbourg",
"Colmar", "Eguisheim", "Besancon", "Dijon", "Annecy", "Grenoble", "Lyon", "Gorges du Verdon", "Bormes les Mimosas", "Cassis", "Marseille", "Aix en Provence",
"Avignon", "Uzes", "Nimes", "Aigues Mortes", "Saintes Maries de la mer", "Collioure", "Carcassonne", "Ariege", "Toulouse", "Montauban", "Biarritz",
"Bayonne", "La Rochelle"]

Creating our own dataframe using two APIs : the first one for each city's location and the other one for the weather predictions :

In [ ]:
meteo_list = []
for city in cities:
    city_requests = requests.get(f'https://nominatim.openstreetmap.org/?q={city}&countrycodes=FR&format=json')
    lat_city = float(city_requests.json()[0]['lat'])
    lon_city = float(city_requests.json()[0]['lon'])
    weather_forecast = requests.get(f'https://api.openweathermap.org/data/2.5/forecast?lat={lat_city}&lon={lon_city}&appid=683cc49f15c0d348d32dde05c7f62442&units=metric')
    sum_pop = sum([weather_forecast.json()['list'][i]['pop'] for i in range(40)])
    meteo_list.append([city,lat_city,lon_city,sum_pop])

meteo_df = pd.DataFrame(meteo_list, columns=['city','lat_city','lon_city','sum_pop'])
meteo_df

,city,lat_city,lon_city,sum_pop
0,Mont Saint Michel,48.635954,-1.511460,10.91
1,St Malo,48.649518,-2.026041,11.93
2,Bayeux,49.276462,-0.702474,8.61
3,Le Havre,49.493898,0.107973,8.40
4,Rouen,49.440459,1.093966,7.78
5,Paris,48.858890,2.320041,6.07
6,Amiens,49.894171,2.295695,6.41
7,Lille,50.636565,3.063528,6.43
8,Strasbourg,48.584614,7.750713,5.36
9,Chateau du Haut Koenigsbourg,48.249490,7.344296,5.97


## 🗺️ Creating a weather map of France

In [ ]:
import plotly.express as px
fig = px.scatter_mapbox(meteo_df, lat="lat_city", lon="lon_city", color="sum_pop", center=dict( lat=47, lon=-0.8), zoom=4, hover_name="city", size="sum_pop", mapbox_style="stamen-toner", color_continuous_scale="Blues")
fig.update_layout(
        title_text = 'Précipitations à prévoir',
        mapbox={"center": {"lon": 2, "lat" : 47}, "zoom": 4.5},
        margin={"l": 380, "r": 380, "b": 0, "t": 0}
        )
    
fig.show()

In [ ]:
best_cities = meteo_df.groupby('city')['sum_pop'].mean().sort_values().index
print(f'Les cinq villes où il pleuvra le moins dans les cinq prochains jours sont : {best_cities[0]}, {best_cities[1]}, {best_cities[2]}, {best_cities[3]} et {best_cities[4]}.')

Les cinq villes où il pleuvra le moins dans les cinq prochains jours sont : Marseille, Cassis, Aix en Provence, Avignon et Montauban.


## 🏨 Scrape Booking.com

In [ ]:
import os 
import logging
import scrapy
from scrapy.crawler import CrawlerProcess
import warnings
warnings.simplefilter("ignore")

class BookingSpider(scrapy.Spider):
    
    name = "booking"
    start_urls = ['https://www.booking.com/']
    

    def parse(self, response):
        city_list = ["Mont Saint Michel", "St Malo", "Bayeux", "Le Havre", "Rouen", "Paris", "Amiens", "Lille", "Strasbourg", "Chateau du Haut Koenigsbourg",
"Colmar", "Eguisheim", "Besancon", "Dijon", "Annecy", "Grenoble", "Lyon", "Gorges du Verdon", "Bormes les Mimosas", "Cassis", "Marseille", "Aix en Provence",
"Avignon", "Uzes", "Nimes", "Aigues Mortes", "Saintes Maries de la mer", "Collioure", "Carcassonne", "Ariege", "Toulouse", "Montauban", "Biarritz",
"Bayonne", "La Rochelle"]
        for city in city_list:            
            yield scrapy.FormRequest.from_response(
            response,
            formdata={'ss': city},
            callback=self.after_search,
            meta={'city': city}
         )

       
    def after_search(self, response):  
        urls = response.xpath('//*[@id="search_results_table"]/div[2]/div/div/div/div[5]/div/div[1]/div[2]/div/div/div[1]/div/div[1]/div/h3/a')
        urls = urls[0:20]      
        for url in urls:
            yield response.follow(url, self.parse_hotel, meta={'city': response.meta['city']})
               
    
    def parse_hotel(self, response):
        yield { 'city': response.meta['city'],             
                'name': response.xpath('//*[@id="hp_hotel_name"]/div/div/h2/text()').get(),
                'note': response.xpath('//*[@id="js--hp-gallery-scorecard"]/a/div/div/div/div/div[1]/text()').get(),
                'loc': response.xpath('//*[@id="b2hotelPage"]/script[26]/text()').get(),
                'description': response.xpath('//*[@id="property_description_content"]/p/text()').get(),
                'url': response.request.url}

filename = "booking.json"

if filename in os.listdir('src/'):
        os.remove('src/' + filename)


process = CrawlerProcess(settings = {
    'USER_AGENT': 'Chrome/97.0',
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        'src/' + filename : {"format": "json"},
    }
})


process.crawl(BookingSpider)
process.start()

2022-12-15 13:29:07 [scrapy.utils.log] INFO: Scrapy 2.7.0 started (bot: scrapybot)
2022-12-15 13:29:07 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 2.0.1, Twisted 22.8.0, Python 3.10.8 (tags/v3.10.8:aaaf517, Oct 11 2022, 16:50:30) [MSC v.1933 64 bit (AMD64)], pyOpenSSL 22.1.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 38.0.1, Platform Windows-10-10.0.19044-SP0
2022-12-15 13:29:07 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 20, 'USER_AGENT': 'Chrome/97.0'}
2022-12-15 13:29:08 [scrapy.extensions.telnet] INFO: Telnet Password: bb6ae1e37f7bbc46
2022-12-15 13:29:09 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2022-12-15 13:29:12 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy

### Creating a new dataframe with hotels' description

In [ ]:
hotels = json.load(open(r'C:\Users\PC\Downloads\jedha\kayak\src\booking.json'))

In [ ]:
hotels_df = pd.DataFrame.from_dict(hotels)

### Merging the two dataframes

In [ ]:
merged_df = meteo_df.merge(hotels_df, on='city')
merged_df.dropna(inplace=True)
merged_df['lat_hotel'] = pd.to_numeric(merged_df['loc'].str.split('latitude = ').str[1].str.split(';').str[0])
merged_df['lon_hotel'] = pd.to_numeric(merged_df['loc'].str.split('longitude = ').str[1].str.split(';').str[0])
merged_df['note'] = pd.to_numeric(merged_df['note'])
merged_df.drop('loc', axis=1, inplace=True)

In [ ]:
merged_df.head()

,city,lat_city,lon_city,sum_pop,name,note,description,url,lat_hotel,lon_hotel
0,Mont Saint Michel,48.635954,-1.51146,10.91,Hôtel Vert,8.0,You're eligible for a Genius discount at Hôtel...,https://www.booking.com/hotel/fr/vert.en-gb.ht...,48.614700,-1.509617
1,Mont Saint Michel,48.635954,-1.51146,10.91,Mercure Mont Saint Michel,8.2,This Mercure is situated in parkland just 2 km...,https://www.booking.com/hotel/fr/mont-saint-mi...,48.614247,-1.510545
2,Mont Saint Michel,48.635954,-1.51146,10.91,La Mère Poulard,7.0,You're eligible for a Genius discount at La Mè...,https://www.booking.com/hotel/fr/la-mere-poula...,48.635085,-1.510540
3,Mont Saint Michel,48.635954,-1.51146,10.91,Les Terrasses Poulard,7.3,You're eligible for a Genius discount at Les T...,https://www.booking.com/hotel/fr/les-terrasses...,48.635349,-1.510379
4,Mont Saint Michel,48.635954,-1.51146,10.91,Le Relais Saint Michel,7.8,You're eligible for a Genius discount at Le Re...,https://www.booking.com/hotel/fr/le-relais-sai...,48.617587,-1.510396


## 🗺️ Creating a map of France with the best cities' hotels

In [ ]:
best_cities_df = merged_df[merged_df['city'].isin(best_cities[0:5])]

In [ ]:
import plotly.express as px
fig = px.scatter_mapbox(best_cities_df, lat="lat_hotel", lon="lon_hotel", color="note", center=dict( lat=47, lon=-0.8), zoom=4, hover_name="city", size="note", mapbox_style="stamen-toner", color_continuous_scale="Reds")
fig.update_layout(
        title_text = 'Précipitations à prévoir',
        mapbox={"center": {"lon": 2, "lat" : 47}, "zoom": 4.5},
        margin={"l": 280, "r": 280, "b": 0, "t": 0}
        )
    
fig.show()

## ☁️ Creating our data lake using S3

In [ ]:
!pip install Boto3 -q


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import boto3
session = boto3.Session(aws_access_key_id="XXXXXX", 
                        aws_secret_access_key="XXXXXX")

In [ ]:
s3 = session.resource("s3")

In [ ]:
bucket = s3.create_bucket(Bucket="bucket-for-kayak-marie-jedha")

In [ ]:
csv = merged_df.to_csv(index=False)

In [ ]:
put_object = bucket.put_object(Key='merged_df.csv', Body=csv)

## 🧹 ETL 

In [ ]:
!pip install pymysql -q
!pip install psycopg2-binary -q


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker

In [ ]:
DBHOST = "kayak.cllpvnkjlqgv.us-east-1.rds.amazonaws.com"
DBUSER = "XXXXXX" 
DBPASS = "XXXXXX"
DBNAME = "postgres"
PORT = "5432"

In [ ]:
engine = create_engine(f"postgresql+psycopg2://{DBUSER}:{DBPASS}@{DBHOST}/{DBNAME}", echo=True)

In [ ]:
Session = sessionmaker(bind = engine)
session = Session()

In [ ]:
merged_df.to_sql("kayak_sql", engine)
# I didn't manage this final part of the project. I think there's somathing wrong about the security groups of my RDS instance but I didn't find the right parameters


KeyboardInterrupt

